In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn

每週資料加總

In [2]:
def week_data_sum(data,year):
#     刪除第53週
    data = data.drop(data[data['週'] == 53].index)
    part_compute = pd.DataFrame()
    total_sum = 0
    for num in range(1, 53,2):
        temp_1 = data[data['週'] == num]
        temp_2 = data[data['週'] == num+1]
        last_temp = temp_1.tail(1)
        print("檢查",temp_1['休假天數'])
        vacation = temp_1['休假天數'].values[0] + temp_2['休假天數'].values[0]
        work =  temp_1['工作天數'].values[0] + temp_2['工作天數'].values[0]
        last_temp['休假天數'] = vacation
        last_temp['工作天數'] = work
        print(vacation)
        if len(temp_1) == 0 & len(temp_2) == 0:
            last_temp['數量'] = 0
            last_temp['週'] = num
            last_temp['年'] = year
            part_compute = part_compute.append(last_temp)
        else:
            temp_1 = temp_1.drop(temp_1[temp_1['數量'] < 0].index)
            temp_2 = temp_2.drop(temp_2[temp_2['數量'] < 0].index)
            last_temp['數量'] =  temp_1['數量'].sum() + temp_2['數量'].sum() 
            part_compute = part_compute.append(last_temp)
        total_sum = total_sum +part_compute['數量'].values[0]
    print(part_compute)
    print(total_sum)
    
    return part_compute


重新計算工作日

In [3]:
def work_day(data,year):
    
#     先刪除工作日和休假日
    data = data.drop(['工作天數', '休假天數'], axis=1)
    
    check_date = pd.read_csv(f"../使用量計算/週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
  
#     answer = pd.merge(data, date, on='週',how='outer')

    answer = pd.merge(data, date, on='週')
#     answer = answer.sort_values(['帳務日期'], ascending=True).reset_index(drop=True)

#     刪除日期和工作日
    answer.rename(columns={'年_x': '年'}, inplace=True)
    print("查看合併",answer)
    return answer


資料集不足的週數補齊

In [4]:
def Compensation(data,start_yeat,end_year):
#      保存缺失的值
    part_compute = pd.DataFrame()
    last_temp = data.head(1)
    for year in range(start_yeat, end_year+1,1):
        for num in range(1, 53,1):
            temp = data[(data['週'] == num)&(data['年'] == year)]
            if temp.empty == True:
                last_temp['數量'] = 0
                last_temp['週'] = num
                last_temp['年'] = year
                part_compute = part_compute.append(last_temp)
    total_data = pd.concat([data, part_compute])
    print("長度",len(total_data))
    
    return total_data


進行差值

In [5]:
def interpolation(data):

#     1.先判斷是否有0的資料，將0的資料改成nan np.nan
    data.loc[data['數量']==0,'數量'] = np.nan
#     2.利用插值套件進行線性插值，插值套件會將nan的值進行插值
    data['數量'] = data['數量'].interpolate(method = 'polynomial', order = 2).round(decimals = 2)
    print("更改數值:",len(data) )

    return data


提取前n天的資料

In [6]:
# 總資料data
# 要提取前幾天資料n_day
def take_data(data,n_day):
    data_copy = data.copy()
    # 總共要提取幾次
    for num in range(n_day, len(data_copy),1):
#         每次提取n筆
        for time in range(1,n_day+1,1):
            before_day = data_copy.loc[num-time:num-time,"數量"].round(decimals = 2)
            print(type(before_day))
            data_copy.loc[num:num,f'前{time}週'] = before_day.values[0]

    return data_copy

過年值特別處理

In [7]:
# 2019年過年 2月2日~2月10日
# 2020年過年 1月23日~1月29日
# 2021年過年 2月10日~2月16日
# 2022年過年 1月31日~2月6日
# 2021年2月20日雖然為假日但為補班日
# 先計算上述時間是第幾週，並判斷該週休假日是否大於等於4天
def new_year(data):
    date = pd.DataFrame()
    new_year = [[2019,2,2,2019,2,10],[2020,1,23,2020,1,29],[2021,2,10,2021,2,16],[2022,1,31,2022,2,6]]
    for i in range(len(new_year)):
        date = pd.DataFrame()
        date['日期'] = pd.DataFrame(pd.date_range(f'{new_year[i][1]}/{new_year[i][2]}/{new_year[i][0]}',f'{new_year[i][4]}/{new_year[i][5]}/{new_year[i][3]}'))
        date['週'] = date['日期'].dt.isocalendar().week
        date.drop_duplicates(subset='週', keep='last', inplace=True)
       
    #     選出該週並判斷該週的休假日是否大於等於4
        for week in range(len(date)):
            target = data[(data["年"] == new_year[i][0]) & (data["週"] == date['週'].values[week])]
            target_index = data[(data["年"] == new_year[i][0]) & (data["週"] == date['週'].values[week])].index.astype(int)
#             如果沒有那週的資料則用去年的資料補上並做更改
            if target.empty:
                temp = data[(data["年"] == new_year[i][0]-1) & (data["週"] == date['週'].values[week])]
                temp['年'] = 2022
                temp['數量'] = 0
                temp['休假天數'] = 7
                temp['工作天數'] = 0
                data = data.append(temp,ignore_index=True)
                target = data[(data["年"] == new_year[i][0]) & (data["週"] == date['週'].values[week])]
                target_index = data[(data["年"] == new_year[i][0]) & (data["週"] == date['週'].values[week])].index.astype(int)

#                 找出2019年第五周的資料並將數量改成0，年改成2022
                
            if target['休假天數'].values[0] >= 4:
                data.loc[target_index.values[0]:target_index.values[0],'過年'] = True
#         找出有過年的那幾週的資料
    data = data[data.columns.drop(list(data.filter(regex='level_0')))]
    new_year_day = data[data['過年'] == True].reset_index()
#     print('-----------------------------',new_year_day)
  

#     找出同一年重複的，只留最少的
    for i in range(len(new_year)):
        if len(new_year_day[new_year_day['年'] == new_year[i][0]]) > 1:
#             找出不要的
            temp1 = data[(data['年']== new_year[i][0]) & (data['過年']==True)]
            temp1_index = temp1[temp1['數量'] == temp1['數量'].max()].index.values[0]
            data.loc[temp1_index:temp1_index,"過年"] = False
            temp2= new_year_day[new_year_day['年']== new_year[i][0]]
            temp2_index =temp2[temp2['數量'] == temp2['數量'].max()].index.values[0]
            new_year_day = new_year_day.drop(temp2_index)
    
#     新增使用量欄位
    new_year_day = data[data['過年'] == True].reset_index()
    new_year_day_index = data[data['過年'] == True].index.astype(int)
    for i in range(len(new_year_day)):
        if i == 0:
            before_day = data.loc[new_year_day_index[i]-1:new_year_day_index[i]-1,"數量"].round(decimals = 2)
        else:
            before_day = new_year_day.loc[i-1:i-1,'數量'].round(decimals = 2)
    #         data.loc[new_year_day_index[i]:new_year_day_index[i],f'使用量(過年)'] = before_day.values[0]
        new_year_day.loc[i:i,'使用量(過年)'] = before_day.values[0]
#     刪除過年資料並且另外處理計算
    data = data.drop(data[data['過年']==True].index).reset_index(drop=True)

    
    return data,new_year_day



評估績效

In [8]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def nMAE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))/y_true.mean() * 100

def RMSE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(((y_pred - y_true) ** 2).mean())

將過年刪除的資料放回test資料集

In [9]:
def put_data_back(y_test,y_pred,new_year_day,test_year):
    y_test_copy = y_test.copy()
    y_pred = pd.DataFrame(y_pred,columns=['前1週'])
    y_pred_copy = y_pred.copy()
#     算出他維資料集中的第幾筆
    new_year_day = new_year_day[new_year_day['年'] == test_year]
    y_pred_copy = y_pred_copy.reset_index(drop = True)
    y_test_copy = y_test_copy.reset_index()
    y_test_copy = y_test_copy[y_test_copy.columns.drop(list(y_test_copy.filter(regex='index')))]
#     插入被刪除過年資料(y_test)
    df_part_1 = y_test_copy.loc[0:3]
    df_part_2 = y_test_copy.loc[4:]
#     print(df_part_2)

    y_test = df_part_1['數量'].append(new_year_day['數量'],ignore_index = True)
    y_test = y_test.append(df_part_2['數量'],ignore_index = True)
  #     插入被刪除過年資料(y_pred)
    df_part_1_pred = y_pred_copy.loc[0:3]
    df_part_2_pred = y_pred_copy.loc[4:]
#     print(df_part_2)

    y_pred = df_part_1_pred['前1週'].append(new_year_day['使用量(過年)'],ignore_index = True)
    y_pred = y_pred.append(df_part_2_pred['前1週'],ignore_index = True)
    return y_test,y_pred

In [10]:
def work_day(data,year):
#     先刪除工作日和休假日
    data = data.drop(['工作天數', '休假天數'], axis=1)
    
    check_date = pd.read_csv(f"../使用量計算/週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
  
#     answer = pd.merge(data, date, on='週',how='outer')

    answer = pd.merge(data, date, on='週')
#     answer = answer.sort_values(['帳務日期'], ascending=True).reset_index(drop=True)

#     刪除日期和工作日
    answer.rename(columns={'年_x': '年'}, inplace=True)
    
    return answer

讀取資料，切割資料集

In [11]:
# B03106859
# C13930399
# A00120213
# A02120180
# A02320340
# A02322259
# A04411285
# A04800036
# B00206057 50週
# B03110120
# G81500272
# K80004044
room = "五病房"
part_no = 'A04411285'

# part_no_total = ["J00202019","T01101024","B00204053","Y12876111","C00211075","U00202042","C13941807","V00400273","A00120033","A00120110","Y12876309"
#                  ,"B08143445","I20100104","A04411370","K82430329","C00211067","C02514164","C00120142","F36005006"]

total_answer_1 = pd.DataFrame()
for part_no in part_no_total:
    before_n = 1

    data=pd.read_csv(f"../使用量計算/有加工作日/差值測試/{room}/{part_no}.csv")
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))]
#     data['年'] =  pd.to_datetime(data['帳務日期']).dt.year

 # 補其不足的週數
    data = Compensation(data,2019,2022)
    data_2019 = data[data['年']==2019]
    data_2019 = work_day(data_2019,2019)
    data_2020 = data[data['年']==2020]
    data_2020 = work_day(data_2020,2020)
    data_2021 = data[data['年']==2021]
    data_2021 = work_day(data_2021,2021)
    data_2022 = data[data['年']==2022]
    data_2022 = work_day(data_2022,2022)
    
  # 把每一週的使用量加總
    data_2019 = week_data_sum(data_2019,2019)
    data_2020 = week_data_sum(data_2020,2020)
    data_2021 = week_data_sum(data_2021,2021)
    data_2022 = week_data_sum(data_2022,2022)
    total_data = pd.concat([data_2019, data_2020])
    total_data = pd.concat([total_data, data_2021])
    total_data = pd.concat([total_data, data_2022])
    total_data = total_data.reset_index()
    #     判斷是否為過年週如果不是並且使用量是0的情況底下，進行差值的運算
    total_data = interpolation(total_data)
    
    total_data = total_data.reset_index()
    total_data = take_data(total_data,before_n)
    total_data.drop(total_data.head(before_n).index,inplace=True) # 从头去掉n行
    total_data['帳務日期'] =  pd.to_datetime(total_data['帳務日期'])
    print("資料檢查",total_data)
    test_data = total_data[total_data['年']== 2022]  
    y_test = test_data["數量"]
    y_pred = test_data['前1週']
  

#     mape = round(MAPE(y_test, y_pred),2)
    rmse = round(RMSE(y_test, y_pred),2)
    mae = round(nMAE(y_test, y_pred),2)
    pred_result = pd.DataFrame({'料號': part_no,'P(RMSE)': rmse,'P(Mae)': mae},index=[0])
    total_answer_1 = total_answer_1.append(pred_result,ignore_index=True)
total_answer_1

C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3269389957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_temp['數量'] = 0
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3269389957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_temp['週'] = num
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3269389957.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

長度 235
     level_0  index         日期   週     年    休假日
0          0      0 2019-01-01   1  2019   True
1          1      1 2019-01-02   1  2019  False
2          2      2 2019-01-03   1  2019  False
3          3      3 2019-01-04   1  2019  False
4          4      4 2019-01-05   1  2019   True
..       ...    ...        ...  ..   ...    ...
358      358    358 2019-12-25  52  2019  False
359      359    359 2019-12-26  52  2019  False
360      360    360 2019-12-27  52  2019  False
361      361    361 2019-12-28  52  2019   True
362      362    362 2019-12-29  52  2019   True

[363 rows x 6 columns]
     level_0  index         日期   週     年    休假日
0          0      0 2020-01-01   1  2020   True
1          1      1 2020-01-02   1  2020  False
2          2      2 2020-01-03   1  2020  False
3          3      3 2020-01-04   1  2020   True
4          4      4 2020-01-05   1  2020   True
..       ...    ...        ...  ..   ...    ...
357      357    357 2020-12-23  52  2020  False
358      

C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3290689985.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3290689985.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3290689985.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\I

     level_0  index         日期   週     年    休假日
0          0      3 2021-01-04   1  2021  False
1          1      4 2021-01-05   1  2021  False
2          2      5 2021-01-06   1  2021  False
3          3      6 2021-01-07   1  2021  False
4          4      7 2021-01-08   1  2021  False
..       ...    ...        ...  ..   ...    ...
357      357    360 2021-12-27  52  2021  False
358      358    361 2021-12-28  52  2021  False
359      359    362 2021-12-29  52  2021  False
360      360    363 2021-12-30  52  2021  False
361      361    364 2021-12-31  52  2021   True

[362 rows x 6 columns]
     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    

C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

6.0
檢查 10    2.0
Name: 休假天數, dtype: float64
4.0
檢查 28    2.0
Name: 休假天數, dtype: float64
5.0
檢查 30    2.0
Name: 休假天數, dtype: float64
4.0
檢查 32    2.0
Name: 休假天數, dtype: float64
4.0
檢查 12    3.0
Name: 休假天數, dtype: float64
5.0
檢查 35    2.0
Name: 休假天數, dtype: float64
4.0
檢查 37    2.0
Name: 休假天數, dtype: float64
4.0
檢查 39    2.0
Name: 休假天數, dtype: float64
4.0
檢查 41    2.0
Name: 休假天數, dtype: float64
4.0
檢查 43    2.0
Name: 休假天數, dtype: float64
4.0
檢查 45    2.0
Name: 休假天數, dtype: float64
5.0
檢查 47    3.0
Name: 休假天數, dtype: float64
5.0
檢查 49    2.0
Name: 休假天數, dtype: float64
4.0
檢查 50    4.0
Name: 休假天數, dtype: float64
6.0
檢查 52    2.0
Name: 休假天數, dtype: float64
4.0
檢查 54    2.0
Name: 休假天數, dtype: float64
4.0
檢查 56    2.0
Name: 休假天數, dtype: float64
4.0
檢查 16    2.0
17    2.0
Name: 休假天數, dtype: float64
4.0
檢查 18    2.0
Name: 休假天數, dtype: float64
4.0
      庫房         料號     品名   數量        帳務日期   週    工作日     年   年_y  工作天數  休假天數
21  DLN5  C14000004  輸液延長管  0.0  2019-02-06   1  False  2019  2019   8.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_temp['休假天數'] = vacation
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_temp['工作天數'] = work
C:\Users\IDSL\AppData\Local\Temp\ipykernel_17024\3423363697.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

更改數值: 104
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.cor

,料號,P(RMSE),P(Mae)
0,C14000004,NaN,NaN


In [12]:
total_answer_1

,料號,P(RMSE),P(Mae)
0,C14000004,NaN,NaN
